In [33]:
import os
import time
from dotenv import load_dotenv
from langchain_text_splitters import CharacterTextSplitter
from pinecone import Pinecone
#from langchain.document_loaders import TextLoader
#from langchain.embeddings import HuggingFaceEmbeddings

##loading embeddings to Pinecone
#from langchain.vectorstores import Pinecone

In [ ]:
load_dotenv()
key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=key)
# name cannot be underscore
index_name = "chatbot-demo"

In [34]:
##2.Loading the embeddings to our PineCone Client


#3.checking if index exists
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name = index_name,
        cloud = "aws",
        region = "us-east-1",
        embed = {
            "model" : "llama-text-embed-v2",
            "field_map" : {"text":"chunk_text"}
        }
    )
    while not pc.describe_index(index_name).index.status['ready']:
        time.sleep(1)
    print("Pinecone Index provisioned")
else:
    print("Pinecone Index Already provisioned")

Pinecone Index Already provisioned


In [37]:
# Use the already imported TextLoader and CharacterTextSplitter if you want to split by character or sentence.
# If you want to split by sentence, you can use CharacterTextSplitter with appropriate settings.

file_path = "./sample.txt"
document_loader = TextLoader(file_path)
documents = document_loader.load()

# If you want to split by sentences, you can use CharacterTextSplitter with separator as '.' or '\n'
# Here, let's use CharacterTextSplitter to split by sentences (approximate, for demo)
text_splitter = CharacterTextSplitter(separator='.', chunk_size=1, chunk_overlap=0)

for document in documents:
    for sentence in text_splitter.split_text(document.page_content):
        print(sentence.strip())

Created a chunk of size 65, which is longer than the specified 1
Created a chunk of size 19, which is longer than the specified 1
Created a chunk of size 149, which is longer than the specified 1
Created a chunk of size 246, which is longer than the specified 1
Created a chunk of size 27, which is longer than the specified 1
Created a chunk of size 140, which is longer than the specified 1
Created a chunk of size 330, which is longer than the specified 1
Created a chunk of size 116, which is longer than the specified 1
Created a chunk of size 78, which is longer than the specified 1
Created a chunk of size 138, which is longer than the specified 1
Created a chunk of size 12, which is longer than the specified 1
Created a chunk of size 215, which is longer than the specified 1
Created a chunk of size 65, which is longer than the specified 1
Created a chunk of size 104, which is longer than the specified 1
Created a chunk of size 106, which is longer than the specified 1
Created a chunk 

There Was an Idea
The Avengers Initiative was the brainchild of S
H
I
E
L
D
Director Nick Fury
He first approached Tony Stark with the idea, following Tonyâ€™s defeat of Obadiah Stane and his subsequent public announcement that he was Iron Man
Fury kept his eye on several potential members, as Bruce Banner struggled with life as the Hulk, the Asgardian Thor appeared on Earth, and Steve Rogers, AKA World War II hero Captain America, was discovered alive decades after his apparent death
In the meantime, some of S
H
I
E
L
D
â€™s most skilled members, Black Widow (Natasha Romanoff), and Hawkeye (Clint Barton), were making a name for themselves and impressing Fury
Once the Asgardian trickster god Loki arrived on Earth and absconded with the massively powerful object known as the Tesseractâ€”possessing Hawkeye and turning him into his brainwashed pawn in the processâ€”Fury found it necessary to finally bring together his potential super team, calling in Romanoff, Banner, Rogers and Stark
Cap

In [39]:


##upsert text into a namespace
##whats a dense index
dense_index = pc.Index(index_name)
#upser the recors into a namespace
dense_index.upsert_records("example-namespace",documents)


AttributeError: 'Document' object has no attribute 'get'